## Langchain demo of simple sequential chains

In class, Alex showed y'all basic simple sequential chains, we will buid on top of this today.
First, let's get all the config and imports stuff in order.

1. Notice the the below code block has `import config`, find the `config.py` file (in the same directory) and put your openai API key in here.  If you do not have one yet, create a new account and refer to [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) to find your openai API key. 

2. Once you have your key, update the key in `config.py`, it should have the format of `sk-XXXXXXXXXXXXXXXXXX` (number of X's is not exact so don't take that literally please).

3. Run the block below once the above 2 are done so setup the environment.

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub, LLMChain
from langchain.chains import SequentialChain, SimpleSequentialChain

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
import config
import os

os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

# uncomment below if you want to use huggingface instead, when you run this, you will see an input box below, paste in your huggingface token in there
# and wait to see "........"
# from getpass import getpass
# HUGGINGFACEHUB_API_TOKEN = getpass()
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

### What do we know?

We can build a chain by piping the output from 1 prompt into the input of another.  We saw this in class with poems and analysis.

The poem template was as such:
```
poem_template = """
You are a Poet. Given the title of a poem, write a haiku for that title.
Haiku for: 
{title}
"""

and analysis template as such:
```
analysis_template = """You are a poet critic. Given a haiku, it is your job to write an analysis.
Poem analysis for:
{haiku}
"""


And we chained them together like this: `overall_chain = SimpleSequentialChain(chains=[poem_chain, analysis_chain], verbose=True)`

Now this is pretty cool, but there is something subtle, see if you can spot it.
Hint: it has something to do with the initial input variable: title, where did it go?``
```
o do so.

#### The problem

If you have not figured it out already, our initial `title` variable is missing from the output!
In fact, imagine a chain as such

foo -> bar -> baz

where foo takes input a
bar takes input b
baz takes input c

so it becomes foo(a) -> bar(b = foo(a)) -> baz(c = bar(foo(a)))

Since every chain effectively ***consumes*** the input, that input is lost forever (or is it??) from the final output.  If you run the chain discussed in class, all we get is the analysis of the poem, we miss the poem title and the poem itself.

We will recreate the situation below with food and restaurants.

In [2]:
# Set up the model with temperature
# temperature is a measure of how creative we want the model to be
temperature = 0.5
model = OpenAI(temperature=temperature)

# uncomment below to test you have credits, if not go back up and use huggingface
name = model("say hi")

# uncomment below to use huggingface
# repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
# hmodel = HuggingFaceHub(
#     repo_id=repo_id, model_kwargs={"temperature": temperature, "max_length": 64}
# )

ValidationError: 1 validation error for HuggingFaceHub
__root__
  Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass  `huggingfacehub_api_token` as a named parameter. (type=value_error)

In [11]:
prompt_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "Suggest a fancy name for a restaurant that serves {cuisine} food."
)

# change to hmodel if you want to use huggingface
name_chain = LLMChain(llm=hmodel, prompt=prompt_name, output_key="restaurant_name")

prompt_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}"
)

menu_items_chain = LLMChain(llm=hmodel, prompt=prompt_items, output_key="menu_items")

overall_chain = SimpleSequentialChain(chains=[name_chain, menu_items_chain], verbose=True)
print(overall_chain.run('Mexican'))



> Entering new SimpleSequentialChain chain...
mexican fusion
enchiladas, chile con queso, flautas,

> Finished chain.
enchiladas, chile con queso, flautas,


If you run the above, notice the last line is just the menu items without the restaurant name!

#### Solution

In comes SequentialChains, these are more verbose than SimpleSequentialChains and include more information that we can use in other parts of our code

In [12]:
seq_chain = SequentialChain(
    chains = [name_chain, menu_items_chain], 
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)
seq_chain({'cuisine': 'Chinese'})

{'cuisine': 'Chinese',
 'restaurant_name': 'chinese restaurant',
 'menu_items': 'steamed fish in black bean sauce'}

And you can see, running the above gives us the cuisine, restaurant_name and menu_items as intended!

### Exercise

Below, come up with another sequential chain with >= 3 nodes total, make it different than poems and food and show all output keys.